# 1. Library

In [ ]:
!pip uninstall -y tensorflow
!pip install -q torch diffusers transformers accelerate sentencepiece bitsandbytes dreamsim
!pip install protobuf==3.20.3

In [3]:
from huggingface_hub import login
login("YOUR_HUGGINGFACE_TOKEN")

In [ ]:
!git clone https://github.com/CryAndRRich/insta-flow-edit.git

In [5]:
import sys
sys.path.append("/kaggle/working/insta-flow-edit")

# 2. FlowEdit functions

In [6]:
from flowedit.sd3 import FlowEditSD3
from flowedit.flux import FlowEditFLUX
from flowedit.instaflow import FlowEditInstaFlow

# 3. Model & Data

## 3.1. Configs

In [7]:
from configs.config import Config

In [8]:
Config.set_csv_path("/kaggle/working/insta-flow-edit/data/flowedit_dataset.csv")
Config.set_yaml_path("/kaggle/working/insta-flow-edit/data/flowedit_prompt.yaml")
Config.set_image_dir("/kaggle/working/insta-flow-edit/data/flowedit_images")
Config.set_output_dirs(
    "/kaggle/working/outputs/Source",
    "/kaggle/working/outputs/Target"
)

## 3.2. Load Functions

In [9]:
from utils.load_data import *
from utils.load_models import *

# 4. Run Model

## 4.1. Scripts

In [10]:
from scripts.run import *
from scripts.evaluate import *

## 4.2. Run FlowEdit

In [11]:
if not os.path.exists(Config.CSV_PATH) or not os.path.exists(Config.YAML_PATH):
    print("Warning: Không tìm thấy file .csv hoặc .yaml")

In [ ]:
# Chạy trên 1 ảnh với 1 prompt bất kỳ
MODEL_NAME = "INSTAFLOW" # SD3, FLUX hoặc INSTAFLOW
IMAGE_NAME = "lighthouse"
TARGET_PROMPT_INDEX = 2
test_one(
    MODEL_NAME,
    IMAGE_NAME,
    TARGET_PROMPT_INDEX
)

In [ ]:
# Chạy trên tất cả ảnh trong dataset
MODEL = "INSTAFLOW"
test_all(MODEL)

# 5. Evaluate

## 5.1. Metrics

In [23]:
from utils.load_metrics import *
from scripts.evaluate import calculate_metrics

## 5.2. Report

In [ ]:
output_src_dirs, output_tar_dirs = Config.get_output_dirs()
yaml_path = Config.get_yaml_path()
scores = calculate_metrics(output_src_dirs, output_tar_dirs, yaml_path)

print("=" * 50)
print("KẾT QUẢ ĐÁNH GIÁ")
print("=" * 50)
print(f"Số lượng mẫu: {len(scores['CLIP-T'])}")

print("-" * 16 + "Càng cao càng tốt" + "-" * 17)
print(f"CLIP-T  : {np.mean(scores['CLIP-T']):.6f}")
print(f"CLIP-I  : {np.mean(scores['CLIP-I']):.6f}")
print(f"DINO    : {np.mean(scores['DINO']):.6f}")
print("-" * 16 + "Càng thấp càng tốt" + "-" * 16)
print(f"LPIPS   : {np.mean(scores['LPIPS']):.6f}")
print(f"DreamSim: {np.mean(scores['DreamSim']):.6f}")

print("=" * 50)